## Estableciendo el entorno

Primero que nada, debemos importar las librerías que utilizaremos que, en nuestro caso, serán el módulo `OpenAI` de la librería `openai` y la librería `os`.

Recordemos que la API key que OpenAI debemos guardarla en un lugar seguro y nunca escribirla explícitamente en el código.

In [1]:
from openai import OpenAI
import os

# Cargamos nuestra API de OpenAI (en mi caso, la tengo guardada 
# como una variable de entorno para evitar copiarla aquí de forma explícita)
openai_api_key = os.getenv("OPENAI_API_KEY")

# Inicializar el cliente de OpenAI
llm = OpenAI(api_key = openai_api_key)

# Construyamos nuestro chatbot

## Definiendo el rol de experto
Luego, definimos el tipo de experto que será nuestro chatbot, estableciendo su función a través de un mensaje que luego le pasaremos al modelo de lenguaje:

In [2]:
developer_prompt = {
'role': 'developer',
'content': """
	Eres un asistente amigable y experto en Python. 
	Responde las preguntas de manera clara y concisa.
	"""
}

## El corazón del chatbot

Una de las partes más importantes es la función que se encarga de llamar el modelo de lenguaje en sí. Mediante esta función podremos pasarle a la API y, por lo tanto, al modelo, qué es lo que está pidiendo el usuario, cómo quiere recibir la respuesta, etc. Luego, llama al modelo de lenguaje para que procese su pregunta y devuelve el resultado.

In [3]:
def obtener_respuesta(dev_prompt, mensaje_usuario):
    user_prompt = {
        "role": "user",
        "content": mensaje_usuario
    }
    # Llamada a la API    
    response = llm.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            dev_prompt,
            user_prompt
        ],
    )

    # Retorna el texto generado
    return response.choices[0].message.content

## El programa principal

Una vez que tenemos definida la función que realiza la llamada al LLM, creamos el programa principal, el cual contará con un bucle while para que luego de cada respuesta, el programa nos pida la siguiente pregunta, y así seguir preguntando.

In [ ]:
print("¡Hola! Soy tu asistente en Python. Escribe 'salir' para terminar la conversación.")

while True:
    mensaje_usuario = input("Tú: ")
    if mensaje_usuario.lower() == "salir":
        print("Asistente: ¡Hasta luego!")
        break

    respuesta = obtener_respuesta(developer_prompt, mensaje_usuario)
    print(f"Asistente: {respuesta}")

# Primeros tests

Al testear el programa nos damos cuenta que también nos responde preguntas no relacionadas con Python.
Por eso, debemos modificar el `developer_prompt` para evitar que responda cosas que no están relacionadas con su función. Esto es lo que se le llama un *guardrail* o valla de seguridad (queda mejor en inglés).

In [5]:
developer_prompt = {
    "role": "developer",
    "content": """
        Eres un asistente amigable y experto en Python. 
        Responde las preguntas de manera clara y concisa.
        Solo responde preguntas relacionadas con Python. 
        Comprueba si el pedido se ajusta a algún concepto relacionado con Python.
        Si recibes una pregunta que no esté relacionada en absoluto con Python di que solo conoces sobre Python, nada más.
    """
}